In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.models import load_model
import rhinoMorph
from collections import defaultdict
from tqdm import tqdm, tqdm_pandas

In [100]:
tqdm.pandas()

In [1]:
import tensorflow as tf
tf.__version__
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1274849110062920894
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4154458112
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2607343825132214774
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

# 예측 과정
문장 입력 -> 특수문자 제거 ->
형태소분석기 로드 -> 형태소 분석 -> 토크나이저 로드 -> 토크나이징 -> 패딩 -> 모델 로드 -> 모델 예측 -> 긍/부정 라벨링
                              -> 

In [101]:
maxlen = 1085

class PredictSentence:
    """한 문장을 예측하는 클래스"""
    def __init__(self, model_path:str, tokenizer_path):
        self.model_path = model_path
        self.tokenizer_path = tokenizer_path


    def load(self):
        """모델, 토크나이저, 형태소 분석기 로드 메서드"""
        self.model = load_model(self.model_path)
        print('suceed load_model')
        with open(self.tokenizer_path, 'rb') as file:
            self.tokenizer = pickle.load(file)
        print('suceed load_tokenizer')
        self.rn = rhinoMorph.startRhino()


    def remove_mark(self, sentence: str):
        """특수문자 제거 메서드"""
        sentence_no_mark = sentence
        marks = ["?", "!", "。", "‘", "’", "“", "”", "`", "\'",
        "\"", "(", ")", "{", "}", "[", "]", "─", "『", "』", 
        ",", "ㆍ", "·", "ᆞ", ":", ";", "/", "…", "_", "~", 
        "∼", "∽", "□", "■", "▶", "◀", "◆", "▲", "◇", "◈", 
        "☎", "【", "】", "+", "-", "=", "±", "÷", "×", "*", 
        "^", ">", "<", "｜", "|", "％", "%", "&", "￦", "₩", 
        "\\", "\t", "\r\n", "\n", "＄", "$", "¥", "￥", "£", 
        "￡", "°", "㎞", "㎏", "@", "©", "ⓒ", "↑", "|", "#", 
        "♥", "♡", "★", "☆", "♪", "♬", 'ㅋ', 'ㅠ', 'ㅜ']
        for mark in marks:
            sentence_no_mark = sentence_no_mark.replace(mark, '')
        return sentence_no_mark
    

    def predict(self, sentence: str):
        "한 문장의 긍/부정을 예측하는 메서드"
        self.removed_sentence = self.remove_mark(sentence)
        self.tokened_sentence = self.tokenizer.texts_to_sequences([self.removed_sentence])
        self.padded_sentence = pad_sequences(self.tokened_sentence, maxlen=maxlen, padding='post')
        self.result = self.model.predict(self.padded_sentence)
        self.final_result = self.result
        
        return self.final_result
    
    def apply_pandas(self, data: pd.Series):
        self.applyed_data = data.progress_apply(self.predict)
        return self.applyed_data

    def test_tokenizer(self, sentence:str):
        self.removed_sentence = self.remove_mark(sentence)
        print(self.removed_sentence)
        self.morphed_sentence = rhinoMorph.onlyMorph_list(self.rn, self.removed_sentence, 
                                                          pos=['NNG', 'NNP', 'VV', 'VA', 'XR'], eomi=True)
        print(f'morphed_sentence: {self.morphed_sentence}')
        self.joined_morphed_sentence = ' '.join(self.morphed_sentence)
        print(self.joined_morphed_sentence)

        self.tokened_sentence = self.tokenizer.texts_to_sequences([self.joined_morphed_sentence])
        print(self.tokened_sentence)

        self.padded_sentence = pad_sequences(self.tokened_sentence, maxlen=maxlen, padding='post')
        print(self.padded_sentence)


In [102]:
predictor = PredictSentence(model_path='C:/Users/slugg/Documents/GitHub/three_idiot/model/models/Second_LSTM/10-0.1274.h5', 
                tokenizer_path='C:/Users/slugg/Documents/GitHub/three_idiot/model/Tokenizer/nomark_tokenizer_28153.pickle')

In [103]:
predictor.load()

suceed load_model
suceed load_tokenizer
filepath:  C:\anaconda3\Lib\site-packages
classpath:  C:\anaconda3\Lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


In [104]:
data = pd.read_csv('C:/Users/slugg/Documents/GitHub/three_idiot/data/전처리_완_데이터/review_여성패션.csv', encoding='cp949', index_col=0)
data

,상품명,상품 코드,구매자 평점,리뷰
0,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,땀복 같다 키 근육 형 핏 예쁘다 요 집 입다 벗다 땀 차다 같다 땀 나다 젖다 안...
1,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,평소 사이즈 입다 겨울 껴입다 사이즈 업다 맞다 스판 잇다 활동 하다 모으다 잇다 ...
2,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,신축 성 좋다 핏 예쁘다 평소 사이즈 주문 맞다 핏 작업복 입다 주문 신축 성 좋다...
3,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,완전 이쁘다 편하다 등산복 구입 방 품다 같다 신축 좋다 속 내복 입다 입다 따뜻하...
4,"머렐 MERRELL 여성 기모 스판 본딩 바지, 베이지, 28",7721174639,4,이쁘다 평소 입다 허벅지 생각하다 주문 허리 맞다
...,...,...,...,...
216382,"베이스알파에센셜 여성용 면스판 길이조절 끈나시 3P, 90, 화이트",295583314,5,가격 착하다 끈 길 조절 편하다 기도 하다
216383,"베이스알파에센셜 여성용 면스판 길이조절 끈나시 3P, 95, 화이트",295583314,5,좋다 끈 질 되다 면도 좋다 편하다
216384,"베이스알파에센셜 여성용 면스판 길이조절 끈나시 3P, 90, 화이트",295583314,5,좋다
216385,"베이스알파에센셜 여성용 면스판 길이조절 끈나시 3P, 90, 화이트",295583314,4,입다


In [105]:
print(len(data['리뷰']))
print(len(data['리뷰'].dropna()))

216387
215499


In [106]:
droped_data = data['리뷰'].dropna()
strd_data = droped_data.astype(str)
predict = predictor.apply_pandas(strd_data)
predict

  0%|          | 125/215499 [00:40<19:30:20,  3.07it/s]


KeyboardInterrupt: 

In [23]:
dd = predict.apply(lambda x: round(x[0][0]))
dd

0        1
1        1
2        0
3        0
4        0
        ..
23135    1
23136    1
23137    1
23138    1
23139    1
Name: 리뷰, Length: 23056, dtype: int64

In [25]:
data = data.dropna(subset=['리뷰'])
data['긍/부정 예측'] = dd

C:\Users\slugg\AppData\Local\Temp\ipykernel_11160\3643106245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['긍/부정 예측'] = dd


In [28]:
data.to_csv('C:/Users/slugg/Documents/GitHub/three_idiot/data/전처리_예측/review_패션_유아동패션.csv')

In [12]:
pre = predict.astype(int)
pre

0        0
1        0
2        0
3        0
4        0
        ..
23135    0
23136    0
23137    0
23138    0
23139    0
Name: 리뷰, Length: 23056, dtype: int32

In [ ]:
drop_data = data.dropna(subset=['리뷰'])
drop_data['긍/부정 예측'] = predict
drop_data

In [5]:
predictor.predict(sentence='저희 집에 있는 드라이기가 고장이 났는 지 작동이 되다 안되다 오락가락해서 급구매하게되었습니다. 웬만하며 참고 그냥 쓰려고 했는 데 제가 머리 숱이 많은 편이라서요 그리고 집에 있던 드라이기로 머리를 말리면 머리 속이 많이 간지럽더군요 머리카락도 많이 빠졌구요 이유를 몰랐는 데 제게 맞는 제품들을 알아보다가 드라이기 열로 두피가 자극을 받아서일 수도 있을 것 같아 두보 드라이기를 구매했습니다. 일단 제가 생각한 것보다 드라이기 사이즈도 소리도 굉장히 크게 들렸습니다. 제 머리가 긴편은 아니고 어깨정도 오는 데요 세 아이맘이고 8개월 된 아기가 있어 육퇴후 주로 밤에 머리를 감다보니 머리말리는 데 30~40분 걸려 덜 말리고 자는 경우도 있었어요 두보 드라이기는 확실히 금방 마르고 선풍기 바람 쐬듯하고 두피도 덜 간지럽네요 머리카락도 덜 빠지네요 가장 높은 온도로 설정 후 쿨버튼 키를 누르고 있으면 시원한 바람이 나오는 데 가장 뜨거운 바람과 차가운 바람을 쿨버튼 하나로 조절이 가능해서 색다른 느낌이 드네요 다만 집에 있는 드라이기는 접이식이라 휴대가 간편하거든요 두보는 접이식이 아니라 아쉽네요 그리고 큰 아이가 평상시에 드라이기로 머리 말릴 때 두피가 뜨겁다고 자주 이야기를 했는 데요 두보 드라이기 소리를 듣고 어떤 행동을 보일 지 궁금하네요 좀 더 써보고 다시 리뷰 올리겠습니다. 제 리뷰가 구매하실 때 도움 되시길 바라며 서로 힘이 되는 좋아요~눌러주세요~^^ ')

저희 집에 있는 드라이기가 고장이 났는 지 작동이 되다 안되다 오락가락해서 급구매하게되었습니다. 웬만하며 참고 그냥 쓰려고 했는 데 제가 머리 숱이 많은 편이라서요 그리고 집에 있던 드라이기로 머리를 말리면 머리 속이 많이 간지럽더군요 머리카락도 많이 빠졌구요 이유를 몰랐는 데 제게 맞는 제품들을 알아보다가 드라이기 열로 두피가 자극을 받아서일 수도 있을 것 같아 두보 드라이기를 구매했습니다. 일단 제가 생각한 것보다 드라이기 사이즈도 소리도 굉장히 크게 들렸습니다. 제 머리가 긴편은 아니고 어깨정도 오는 데요 세 아이맘이고 8개월 된 아기가 있어 육퇴후 주로 밤에 머리를 감다보니 머리말리는 데 3040분 걸려 덜 말리고 자는 경우도 있었어요 두보 드라이기는 확실히 금방 마르고 선풍기 바람 쐬듯하고 두피도 덜 간지럽네요 머리카락도 덜 빠지네요 가장 높은 온도로 설정 후 쿨버튼 키를 누르고 있으면 시원한 바람이 나오는 데 가장 뜨거운 바람과 차가운 바람을 쿨버튼 하나로 조절이 가능해서 색다른 느낌이 드네요 다만 집에 있는 드라이기는 접이식이라 휴대가 간편하거든요 두보는 접이식이 아니라 아쉽네요 그리고 큰 아이가 평상시에 드라이기로 머리 말릴 때 두피가 뜨겁다고 자주 이야기를 했는 데요 두보 드라이기 소리를 듣고 어떤 행동을 보일 지 궁금하네요 좀 더 써보고 다시 리뷰 올리겠습니다. 제 리뷰가 구매하실 때 도움 되시길 바라며 서로 힘이 되는 좋아요눌러주세요 
morphed_sentence: ['집', '드라이기', '고장', '나다', '작동', '되다', '안되다', '오락가락', '급구', '매다', '하다', '웬만', '참다', '쓰리다', '머리', '숱', '많다', '편', '라', '집', '드라이기', '머리', '말리다', '머리', '속', '간지럽다', '머리카락', '빠지다', '이유', '모르다', '맞다', '제품', '알아보다', '드라이기', '열다', '두피', '자극', '받다', '같다', '두보', '드라이기', '구매', '

array([[0.99788016]], dtype=float32)

In [6]:
with open('C:/Users/slugg/Documents/GitHub/three_idiot/model/Tokenizer/nomark_tokenizer_28153.pickle', 'rb') as file:
    tokenizer = pickle.load(file)

word_index = tokenizer.word_index
reversed_word_index = {}
for ke, val in word_index.items():
    reversed_word_index[val] = ke

In [12]:
reversed_word_index[801]

'사'

In [8]:
word_index['로켓']

217

# 빈도분석 (Frequency_Analyze)

In [3]:
def read_file(path: str, encoding: str='cp949'):
    with open(path, 'r', encoding=encoding) as file:
        keyword = [line for line in file.readlines()]
    return keyword

keywords = read_file(path='C:/Users/slugg/Documents/GitHub/three_idiot/data/언어사전/빈도분석단어모음_928to315.txt', encoding='euc-kr')
for index, keyword in enumerate(keywords):
    if '\n' in keyword:
        keyword = keyword.replace('\n', '')
        keywords[index] = keyword
    else:
        pass

keywords

['만족',
 '배송',
 '때',
 '크다',
 '빠르다',
 '디자인',
 '가성비',
 '가볍다',
 '예쁘다',
 '이쁘다',
 '바람',
 '소리',
 '성능',
 '저렴',
 '추천',
 '화면',
 '편하다',
 '최고',
 '색상',
 '기능',
 '깔끔',
 '작다',
 '화질',
 '무겁다',
 '바꾸다',
 '사진',
 '소음',
 '크기',
 '무게',
 '불편',
 '굿',
 '미니',
 '연결',
 '고장',
 '배터리',
 '할인',
 '이상',
 '설치',
 '사이즈',
 '버튼',
 '필름',
 '비싸다',
 '선물',
 '튼튼',
 '포장',
 '길다',
 '쉽다',
 '아쉽다',
 '단점',
 '충전',
 '도착',
 '아이',
 '적당',
 '부드럽다',
 '보호',
 '도움',
 '편리',
 '색',
 '블루',
 '박스',
 '용량',
 '친절',
 '화이트',
 '정품',
 '강추',
 '불량',
 '세기',
 '핑크',
 '손목',
 '실버',
 '투명',
 '고급',
 '조용',
 '블랙',
 '마르다',
 '무료',
 '설정',
 '유튜브',
 '얇다',
 '액정',
 '상태',
 '품질',
 '터치',
 '검색',
 '높다',
 '후회',
 '이벤트',
 '감도',
 '전화',
 '교환',
 '온도',
 '무선',
 '작동',
 '적다',
 '적응',
 '싸다',
 '기대',
 '부담',
 '건조',
 '먼지',
 '친구',
 '고정',
 '휴대',
 '전원',
 '힘들다',
 '어렵다',
 '고요',
 '공간',
 '반품',
 '에어',
 '앱',
 '음질',
 '모드',
 '신경',
 '색감',
 '인터넷',
 '예약',
 '컬러',
 '안전',
 '타자',
 '사무',
 '기계식',
 '설명',
 '뜨겁다',
 '깨지다',
 '아프다',
 '나쁘다',
 '기기',
 '실물',
 '인식',
 '서비스',
 '사운드',
 '눌리다',
 '열다',
 '간단',
 '구성',
 '골드',
 '선명',
 '색깔',
 

In [89]:
from typing import Dict, List, Union
class FrequencyAnalyzer:
    """빈도분석 클래스"""
    def __init__(self, key_path: str):
        self.keyword_path = key_path
        self.product_keyword_counts_info = defaultdict(lambda: defaultdict(int))

    def read_file(self, encoding: str='utf-8'):
        with open(self.keyword_path, 'r', encoding=encoding) as file:
            keywords = [line for line in file.readlines()]

        for index, keyword in enumerate(keywords):
            if '\n' in keyword:
                keyword = keyword.replace('\n', '')
                keywords[index] = keyword
            else:
                pass
        
        return keywords
    
    def morph(self, sentence):
        morphed_sentence = rhinoMorph.onlyMorph_list(self.rn, sentence, 
                                                    pos=['NNG', 'NNP', 'VV', 'VA', 'XR'], eomi=True)
        joined_sentence = ' '.join(morphed_sentence)
        return joined_sentence

    def analyze_one_reviews_with_dataframe(self, data: pd.DataFrame) -> Dict[str, Dict[str, Union[str, int]]]:
        """한 제품의 리뷰를 받아 분석하는 메서드"""
        self.data = data
        self.count_stars = [data['사용자 별점'].value_counts()]
        self.prod_num = data['상품 코드'].iloc[0]
        drop_data = data.dropna(subset=['리뷰'])
        self.strd_data = drop_data['리뷰'].astype(str)
        
        for product, group in strd_data:
            total_reviews = len(group)
            positive_group = group[group['평점'] == 1]
            negetive_group = group[group['평점'] == 0]
            total_positive_reviews = len(positive_group)
            total_negetive_reviews = len(negetive_group)

            self.pos_keyword_counts = defaultdict(int)
            self.neg_keyword_counts = defaultdict(int)

            for review in positive_group.iloc[:, 1]:
                for keyword in keywords:
                    if keyword in review:
                        self.pos_keyword_counts[keyword] += 1
            
            for review in negetive_group.iloc[:, 1]:
                for keyword in keywords:
                    if keyword in review:
                        self.neg_keyword_counts[keyword] += 1

            # 상위 5개 긍정 키워드 선택
            top_5_pos_keywords = sorted(self.pos_keyword_counts.items(), key=lambda x: x[1], reverse=True)[:5]

            # 상위 5개 부정 키워드 선택
            top_5_neg_keywords = sorted(self.neg_keyword_counts.items(), key=lambda x: x[1], reverse=True)[:5]

            self.product_keyword_counts_info[product]['total_reviews'] = total_reviews
            self.product_keyword_counts_info[product]['total_positive_reviews'] = total_positive_reviews
            self.product_keyword_counts_info[product]['total_negative_reviews'] = total_negetive_reviews
            self.product_keyword_counts_info[product]['top_5_pos_keywords'] = top_5_pos_keywords
            self.product_keyword_counts_info[product]['top_5_neg_keywords'] = top_5_neg_keywords
            
        return self.product_keyword_counts_info
    
    def analyze_all_reviews_with_dataframe(self, data: pd.DataFrame) -> Dict[str, Dict[str, Union[str, int]]]:
        """모든 제품의 리뷰를 받아 분석하는 메서드"""
        self.data = data
        drop_data = data.dropna(subset=['리뷰']) # 리뷰 없는 행 제거
        drop_data['상품 코드'] = drop_data['상품 코드'].astype(str)

        for product, group in drop_data.groupby('상품 코드'): # 상품명으로 묶어 같은 상품 리뷰 처리
            count_stars = group['구매자 평점'].value_counts().sort_index() # 별점 개수 카운팅
            index = count_stars.index
            ret_dict = {ind: cnt_star for ind, cnt_star in zip(index, count_stars.values.tolist())} # 별점 개수 딕셔너리로 저장
            # prod_num = group['상품 코드'].iloc[0] # 상품 코드 저장
            total_reviews = len(group)
            positive_group = group[group['긍/부정 예측'] == 1]
            negetive_group = group[group['긍/부정 예측'] == 0]
            total_positive_reviews = len(positive_group)
            total_negetive_reviews = len(negetive_group)

            pos_keyword_counts = defaultdict(int)
            neg_keyword_counts = defaultdict(int)

            for review in positive_group['리뷰']:
                for keyword in keywords:
                    if keyword in review:
                        pos_keyword_counts[keyword] += 1
            
            for review in negetive_group['리뷰']:
                for keyword in keywords:
                    if keyword in review:
                        neg_keyword_counts[keyword] += 1

            # 상위 5개 긍정 키워드 선택
            top_5_pos_keywords = sorted(pos_keyword_counts.items(), key=lambda x: x[1], reverse=True)[:5]

            # 상위 5개 부정 키워드 선택
            top_5_neg_keywords = sorted(neg_keyword_counts.items(), key=lambda x: x[1], reverse=True)[:5]

            self.product_keyword_counts_info[product]['total_reviews'] = total_reviews
            self.product_keyword_counts_info[product]['total_positive_reviews'] = total_positive_reviews
            self.product_keyword_counts_info[product]['total_negative_reviews'] = total_negetive_reviews
            self.product_keyword_counts_info[product]['top_5_pos_keywords'] = top_5_pos_keywords
            self.product_keyword_counts_info[product]['top_5_neg_keywords'] = top_5_neg_keywords
            self.product_keyword_counts_info[product]['star_count'] = ret_dict
            # self.product_keyword_counts_info[product]['prod_num'] = prod_num
            
        return self.product_keyword_counts_info

In [90]:
al = FrequencyAnalyzer(key_path='C:/Users/slugg/Documents/GitHub/three_idiot/data/언어사전/패션.txt')

In [91]:
al.read_file(encoding='cp949')

['사이즈',
 '크다',
 '아이',
 '이쁘다',
 '예쁘다',
 '귀엽다',
 '좋아하다',
 '편하다',
 '가격',
 '세탁',
 '괜찮다',
 '재질',
 '디자인',
 '작다',
 '만족',
 '얇다',
 '넉넉',
 '색상',
 '길다',
 '배송',
 '길이',
 '핏',
 '부드럽다',
 '따뜻',
 '색',
 '짧다',
 '겨울',
 '저렴',
 '허리',
 '여아',
 '아들',
 '원단',
 '춥다',
 '불편',
 '여유',
 '보풀',
 '두께',
 '추천',
 '도톰',
 '가성비',
 '소재',
 '한치수',
 '마른',
 '신축',
 '두껍다',
 '늘어나다',
 '적당',
 '줄어들다',
 '아쉽다',
 '색감',
 '활동',
 '색깔',
 '건조',
 '기장',
 '발목',
 '화면',
 '깔끔',
 '선물',
 '반품',
 '줄다',
 '소매',
 '따뜻하다',
 '고무줄',
 '교환',
 '어울리다',
 '컬러',
 '품질',
 '울다',
 '편안',
 '가볍다',
 '고민',
 '덥다',
 '싸이즈',
 '쫀쫀',
 '짱짱',
 '스판',
 '색도',
 '스타일',
 '밴딩',
 '엉덩이',
 '시원',
 '캐릭터',
 '촉감',
 '마감',
 '두툼',
 '먼지',
 '프린팅',
 '무난',
 '작아지다',
 '냄새',
 '싸다',
 '구멍',
 '착용',
 '실밥',
 '실내',
 '퀄리티',
 '밴드',
 '상태',
 '브랜드',
 '날씬',
 '치수',
 '아이보리',
 '착용감',
 '넓다',
 '박음질',
 '체형',
 '강추',
 '변형',
 '바느질',
 '안감',
 '아깝다',
 '화사',
 '크기',
 '흘러내리다',
 '조이다',
 '체격',
 '코디',
 '뻣뻣',
 '옷감',
 '답답',
 '순면',
 '봄',
 '가을',
 '한겨울',
 '프린트',
 '버리다',
 '무릎',
 '허벅지',
 '타이트하다',
 '무늬',
 '다리다',
 '늘어지다',
 '형광',
 '덮다',
 '애매',
 '키우

In [92]:
data = pd.read_csv('C:/Users/slugg/Documents/GitHub/three_idiot/data/전처리_예측/review_패션_유아동패션.csv', encoding='utf-8', index_col=0)
data

,상품명,상품 코드,구매자 평점,리뷰,긍/부정 예측
0,"월튼키즈 남아용 체커보드 플리스 상하의 세트, 블랙, 140",7636639833,5,화면 동일 험하다 다니다 남자아이 한철 입 생각 가격 저렴 구매 만족 세탁 후 어떻...,1
1,"월튼키즈 남아용 체커보드 플리스 상하의 세트, 블랙, 130",7636639833,5,예쁘다 두다 하다 가격 대비 남다 마른 체형 여유 잇지 사이즈 입다 내복 입다 따뜻...,1
2,"월튼키즈 남아용 체커보드 플리스 상하의 세트, 블랙, 130",7636639833,5,살다 맞다 넘다 가 따다,0
3,"월튼키즈 남아용 체커보드 플리스 상하의 세트, 블랙, 120",7636639833,1,빨다 하얗다 부분 검정 물 이염 방지 시트 넣다 빨다 입다 버리다 되다 물들다 디자...,0
4,"마리앤모리 아동용 체크 후리스 기모 상하복 세트, 120, 베이지",7714261280,5,키 몸무게 소재 가 따뜻 색상 이쁘다 서얼 굴 살다 요바 짧다 느낌 있다,0
...,...,...,...,...,...
23135,"메이키즈 아동용 삐약이 7부 쟈가드 실내복, 90호, 옐로우",185480437,5,받다 감사,1
23136,"메이키즈 아동용 삐약이 7부 쟈가드 실내복, 110호, 옐로우",185480437,5,배송 빠르다 옷 넘넘 귀엽다 좋다,1
23137,"메이키즈 아동용 삐약이 7부 쟈가드 실내복, 110호, 옐로우",185480437,4,귀엽다 사이즈 크다 빠르다 배송 굳,1
23138,"메이키즈 아동용 삐약이 7부 쟈가드 실내복, 140호, 옐로우",185480437,5,주문 여유 있다 좋다 화사 예 ㅃ 배송 하루 빠르다,1


In [93]:
analyzed_dict = al.analyze_all_reviews_with_dataframe(data) # star_count는 1점 -> 5점 순으로 개수

In [97]:
analyzed_dict['1230133678']

defaultdict(int,
            {'total_reviews': 3,
             'total_positive_reviews': 2,
             'total_negative_reviews': 1,
             'top_5_pos_keywords': [('귀엽다', 2),
              ('크다', 1),
              ('이쁘다', 1),
              ('사이즈', 1),
              ('색', 1)],
             'top_5_neg_keywords': [('색', 1), ('빨래', 1)],
             'star_count': {2: 1, 5: 2}})

In [98]:
import json

json_string = json.dumps(analyzed_dict, ensure_ascii=False, indent=2)

In [99]:
with open('C:/Users/slugg/Documents/GitHub/three_idiot/data/output/output.json', 'w', encoding='utf-8') as jsfile:
    jsfile.write(json_string)